Run `pip install .` first to install all dependencies.

In [1]:
!pip install gcsfs==2022.10.0
!pip install wandb==0.13.4
!pip install fastparquet
!pip install numpy
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 138 kB 6.5 MB/s 
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.2 MB/s 
     |████████████████████████████████| 168 kB 44.5 MB/s 
     |████████████████████████████████| 182 kB 56.3 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 168 kB 53.8 MB/s 
     |████████████████████████████████| 166 kB 50.5 MB/s 
     |████████████████████████████████| 166 kB 39.6 MB/s 
     |████████████████████████████████| 162 kB 42.6 MB/s 
     |████████████████████████████████| 162 kB 43.3 MB/s 
     |████████████████████████████████| 158 kB 59.5 MB/s 
    

In [5]:
import gcsfs
import numpy as np
import pandas as pd

import wandb

from sklearn.preprocessing import MinMaxScaler


In [3]:
name = "classical_size_features_normalized"

# connect to weights and biases
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")

dataset = wandb.Artifact(name=name, type="preprocessed_data")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
import google.auth
from google.colab import auth
# connect to google cloud storage
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)

In [7]:
# reduce number of imported cols due to memory issues
columns = [
    "QUOTE_DATETIME",
    "ROOT",
    "EXPIRATION",
    "STRK_PRC",
    "OPTION_TYPE",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    "price_all_lead",
    "price_all_lag",
    "price_ex_lead",
    "price_ex_lag",
    "buy_sell",
]


In [8]:
train = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_extended_60.parquet",
    engine="fastparquet",
    columns=columns,
)
val = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_extended_20.parquet",
    engine="fastparquet",
    columns=columns,
)
test = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_extended_20.parquet",
    engine="fastparquet",
    columns=columns,
)


In [9]:
# oe_option_type = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )
# oe_root = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )
# oe_issue_type = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )

min_max_scaler = MinMaxScaler(feature_range=[-1, 1])


def transform(data: pd.DataFrame) -> pd.DataFrame:

    # # date features
    x = pd.DataFrame(data={"TRADE_PRICE": data["TRADE_PRICE"]}, index=data.index)

    # x["date_month_sin"] = np.sin(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)
    # x["date_month_cos"] = np.cos(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)

    # seconds_in_day = 24 * 60 * 60
    # seconds = (
    #     data["QUOTE_DATETIME"] - data["QUOTE_DATETIME"].dt.normalize()
    # ).dt.total_seconds()

    # x["date_time_sin"] = np.sin(2 * np.pi * seconds / seconds_in_day)
    # x["date_time_cos"] = np.cos(2 * np.pi * seconds / seconds_in_day)

    # option features
    # x["ttm"] = (
    #     data["EXPIRATION"].dt.to_period("M") - data["QUOTE_DATETIME"].dt.to_period("M")
    # ).apply(lambda x: x.n)
    # x[["myn", "day_vol"]] = data[["myn", "day_vol"]]
    # x["log_strk_prc"] = np.log1p(data["STRK_PRC"])

    # binarize
    # "bin_OPTION_TYPE", "bin_issue_type", "bin_ROOT",

    # size features
    x["bid_ask_size_ratio_ex"] = data["bid_size_ex"] / data["ask_size_ex"]
    x["rel_bid_size_ex"] = data["TRADE_SIZE"] / data["bid_size_ex"]
    x["rel_ask_size_ex"] = data["TRADE_SIZE"] / data["ask_size_ex"]
    x[["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]] = data[
        ["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]
    ]

    # classical
    mid_ex = 0.5 * (data["ask_ex"] + data["bid_ex"])
    mid_best = 0.5 * (data["BEST_ASK"] + data["BEST_BID"])
    x["rel_ask_ex"] = (data["TRADE_PRICE"] - mid_ex) / (data["ask_ex"] - mid_ex)
    x["rel_bid_ex"] = (mid_ex - data["TRADE_PRICE"]) / (mid_ex - data["bid_ex"])
    x["BEST_rel_bid"] = (data["TRADE_PRICE"] - mid_best) / (data["BEST_ASK"] - mid_best)
    x["BEST_rel_ask"] = (mid_best - data["TRADE_PRICE"]) / (mid_best - data["BEST_BID"])
    x["bid_ask_ratio_ex"] = data["bid_ex"] / data["ask_ex"]

    x["chg_ex_lead"] = data["TRADE_PRICE"] - data["price_ex_lead"]
    x["chg_ex_lag"] = data["TRADE_PRICE"] - data["price_ex_lag"]
    x["chg_all_lead"] = data["TRADE_PRICE"] - data["price_all_lead"]
    x["chg_all_lag"] = data["TRADE_PRICE"] - data["price_all_lag"]

    # x[
    #     [
    #         "log_ask_ex",
    #         "log_bid_ex",
    #         "log_BEST_ASK",
    #         "log_BEST_BID",
    #         "log_trade_price",
    #         "log_price_all_lag",
    #         "log_price_all_lead",
    #         "log_price_ex_lag",
    #         "log_price_ex_lead",
    #     ]
    # ] = np.log1p(
    #     data[
    #         [
    #             "ask_ex",
    #             "bid_ex",
    #             "BEST_ASK",
    #             "BEST_BID",
    #             "TRADE_PRICE",
    #             "price_all_lag",
    #             "price_all_lead",
    #             "price_ex_lag",
    #             "price_ex_lead",
    #         ]
    #     ]
    # )

    x[
        [
            "ask_ex",
            "bid_ex",
            "BEST_ASK",
            "BEST_BID",
            "TRADE_PRICE",
            "price_all_lag",
            "price_all_lead",
            "price_ex_lag",
            "price_ex_lead",
        ]
    ] = data[
        [
            "ask_ex",
            "bid_ex",
            "BEST_ASK",
            "BEST_BID",
            "TRADE_PRICE",
            "price_all_lag",
            "price_all_lead",
            "price_ex_lag",
            "price_ex_lead",
        ]
    ]

    # impute with zeros
    x.replace([np.inf, -np.inf], np.nan, inplace=True)
    x.fillna(0, inplace=True)

    # scale to [-1, 1]
    if not hasattr(min_max_scaler, "n_features_in_"):
        min_max_scaler.fit(x)
    x[x.columns] = min_max_scaler.transform(x)

    # https://stackoverflow.com/questions/70727291/how-do-i-know-whether-a-sklearn-scaler-is-already-fitted-or-not

    # if not hasattr(oe_option_type, "n_features_in_"):
    #     oe_option_type.fit(data["OPTION_TYPE"].astype(str).values.reshape(-1, 1))
    # x["bin_option_type"] = oe_option_type.transform(
    #     data["OPTION_TYPE"].astype(str).values.reshape(-1, 1)
    # )

    # if not hasattr(oe_root, "n_features_in_"):
    #     oe_root.fit(data["ROOT"].astype(str).values.reshape(-1, 1))
    # x["bin_root"] = oe_root.transform(data["ROOT"].astype(str).values.reshape(-1, 1))

    # if not hasattr(oe_issue_type, "n_features_in_"):
    #     oe_issue_type.fit(data["issue_type"].astype(str).values.reshape(-1, 1))
    # x["bin_issue_type"] = oe_issue_type.transform(
    #     data["issue_type"].astype(str).values.reshape(-1, 1)
    # )

    # x.replace([np.inf, -np.inf], np.nan, inplace=True)

    x["buy_sell"] = data["buy_sell"]
    return x


In [10]:
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/train_set_60.parquet"
)
train = transform(train)
train.to_parquet(output_path)
dataset.add_reference(output_path)

del train

In [11]:
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/val_set_20.parquet"
)
val = transform(val)
val.to_parquet(output_path)
dataset.add_reference(output_path)

del val

In [12]:
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/test_set_20.parquet"
)
test = transform(test)
test.to_parquet(output_path)
dataset.add_reference(output_path)


[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/classical_size_features_normalized/test_set_20.parquet/test_set_20.parquet>]

In [13]:
test.describe()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,TRADE_SIZE,bid_size_ex,ask_size_ex,rel_ask_ex,rel_bid_ex,BEST_rel_bid,...,chg_all_lag,ask_ex,bid_ex,BEST_ASK,BEST_BID,price_all_lag,price_all_lead,price_ex_lag,price_ex_lead,buy_sell
count,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,...,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06
mean,-9.923103e-01,-9.998360e-01,-9.998290e-01,-9.998783e-01,-9.996231e-01,-9.978360e-01,-9.975226e-01,-4.496103e-01,4.496103e-01,-2.217289e-01,...,8.878542e-01,-9.919251e-01,-9.926397e-01,-9.999457e-01,-9.925228e-01,-9.989841e-01,-9.998989e-01,-9.931000e-01,-9.946407e-01,-2.805495e-02
std,2.859741e-02,1.158564e-03,1.354047e-03,8.864031e-04,2.769686e-03,5.458831e-03,5.723609e-03,6.287551e-04,6.287550e-04,7.700371e-04,...,5.424119e-04,2.890719e-02,2.823542e-02,1.972331e-04,2.838230e-02,3.795401e-03,3.817389e-04,2.662669e-02,2.121498e-02,9.996064e-01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-5.057560e-01,4.067200e-01,-3.200005e-01,...,7.040394e-01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,-9.995017e-01,-9.999950e-01,-9.999933e-01,-9.999936e-01,-1.000000e+00,-9.998000e-01,-9.997600e-01,-4.499189e-01,4.493566e-01,-2.219640e-01,...,8.878430e-01,-9.993581e-01,-9.996320e-01,-9.999960e-01,-9.995732e-01,-9.999360e-01,-9.999940e-01,-9.997216e-01,-9.998390e-01,-1.000000e+00
50%,-9.981828e-01,-9.999808e-01,-9.999701e-01,-9.999773e-01,-9.999273e-01,-9.994000e-01,-9.993600e-01,-4.495944e-01,4.495944e-01,-2.217396e-01,...,8.878506e-01,-9.979431e-01,-9.983809e-01,-9.999866e-01,-9.983075e-01,-9.997600e-01,-9.999768e-01,-9.985199e-01,-9.989419e-01,-1.000000e+00
75%,-9.944458e-01,-9.999556e-01,-9.998640e-01,-9.998918e-01,-9.996727e-01,-9.982600e-01,-9.979200e-01,-4.493566e-01,4.499189e-01,-2.215077e-01,...,8.878600e-01,-9.938585e-01,-9.948484e-01,-9.999600e-01,-9.947016e-01,-9.992679e-01,-9.999276e-01,-9.951639e-01,-9.963082e-01,1.000000e+00
max,1.832569e+00,-8.631786e-01,-6.186268e-02,-3.209091e-01,7.575229e-01,-7.259173e-01,-7.940379e-01,-4.067197e-01,5.057546e-01,1.930428e-01,...,9.936979e-01,1.834282e+00,1.829555e+00,-9.805971e-01,1.833910e+00,-6.961096e-01,-9.703997e-01,1.226488e+00,1.002507e+00,1.000000e+00


In [14]:
run.log_artifact(dataset)
run.finish()
